# Notebook for analysing roslogs

This notebook analyses the roslog file created which is populated by the rospy.loginfo and rospy.logdebug messages from the algorithm. Point the exp_dir to the root of the experiment directory you want to analyze:

```
exp_dir/
├── progress.csv
├── debug.log
├── variant.json
├── itr_1.pkl
├── itr_2.pkl
.   .   ...
.   .   ...
.   .   ...
└── roslog/
    ├── diayn_env_node_*.log
    └── */

```

In [1]:
%matplotlib inline

import os
import json
import numpy as np
from parse import parse
import matplotlib.pyplot as plt
from collections import OrderedDict

In [2]:
exp_dir = '/localdata2/nikhil/seed_1__snapshot_mode_all__num_skills_10__4fins/'
roslog_dir = 'roslog'
roslog_fileprefix = 'diayn_env_node_'
epoch_filter = []

In [3]:
filepath = None
for dir_item in os.listdir(os.path.join(exp_dir, roslog_dir)):
    if dir_item.startswith(roslog_fileprefix):
        filepath = os.path.join(exp_dir, roslog_dir, dir_item)
print(filepath)

/localdata2/nikhil/seed_1__snapshot_mode_all__num_skills_10__4fins/roslog/diayn_env_node_10231_1524525093018.log


In [4]:
step_return_labels = None
step_return_dict = None
step_epoch = []

if not os.path.exists(filepath):
    print('ERROR: File {} not exist'.format(filepath))
else:
    with open(filepath) as fd:
        for line_num, line in enumerate(fd):
            
            if '[INFO]' in line:
                if 'Epoch #:' and 'Iteration #:' in line:
                    _, epoch, itr = parse('{}: Epoch #: {}, Iteration #: {}', line)
                    epoch = int(epoch)
                    itr = int(itr)
                    
            if epoch_filter == [] or epoch in epoch_filter:
                if '[DEBUG]' in line:
                    if 'reward' in line:

                        _, step_return = parse('{}: {}', line)
                        step_return = json.loads(step_return)

                        if step_return_labels == None:
                            step_return_labels = step_return.keys()
                            step_return_labels.sort()
                            step_return_dict = OrderedDict([(kk, []) for kk in step_return_labels])

                        for kk in step_return.keys():
                            step_return_dict[kk].append(step_return[kk])

                        step_epoch.append(epoch)


#             if line_num == 500: break

In [ ]:
n_plots = len(step_return_labels)

fig_cols = 2
fig_rows = (n_plots / 2) + (n_plots % fig_cols)

n_xticks = 10
xtick_diff = len(step_epoch)/n_xticks
xticks = [step_epoch[ii] for ii in range(len(step_epoch)) if ii%xtick_diff==0]
xtick_locs = [ii for ii in range(len(step_epoch)) if ii%xtick_diff==0]

fig, ax = plt.subplots(fig_rows, fig_cols, figsize=(16, 100))

for col_idx, col in enumerate(step_return_dict.keys()):    
    ax_row = col_idx/fig_cols
    ax_col = col_idx%fig_cols
    ax[ax_row, ax_col].plot(step_return_dict[col])
    
for col_idx, col in enumerate(step_return_dict.keys()):
    ax_row = col_idx/fig_cols
    ax_col = col_idx%fig_cols
    ax[ax_row, ax_col].set_title(col)
    plt.sca(ax[ax_row, ax_col])
    plt.xticks(xtick_locs, xticks)